# **Clase 22-4-2021** 

# Pandas 2020 2C 1

## Enunciado

Una importante compañía de e-commerce acumula información de las interacciones de sus usuarios con su plataforma mediante un motor de analytics. Esta información está disponible en un data frame para que sea analizada por nuestro equipo.


El data frame llamado `events.csv` posee las siguientes columnas:


* uid: identificador único que representa a un usuario que visita la plataforma. 

* sessionid: identificador único de la sesión que generó el usuario.

* event: nombre del evento que se generó. 

* value: información adicional que se guarda con el evento.

* datetime: timedate64 que contiene fecha y hora del evento. 

Para poder realizar un análisis es necesario tener las siguientes consideraciones:


Cada vez que un usuario visita la plataforma, el sistema le genera una nueva sesión con un `sessionid` asociado. Esto implica que un usuario puede realizar múltiples sesiones.


Los eventos son interacciones que realiza el usuario con el sistema. Una sesión está compuesta por al menos un evento.


Se le pide a nuestro equipo las siguientes tareas utilizando pandas:

a) Listar aquellos usuarios que tengan un total de sesiones mayor al promedio de sesiones por usuario del sistema. Ej: si el promedio de los usuarios cuenta con 20 sesiones, se debe listar los usuarios que tengan más de 20 sesiones asociadas. 

b) Calcular el valor promedio de los eventos 'ecommerce.view-product', 'ecommerce.checkout', 'ecommerce.conversion' para aquellos usuarios que hayan realizado más de 25 (tomaremos 3 para el ejercicio) sesiones. Los resultados deben ser presentados en un dataframe con las siguientes columnas ('uid','ecommerce_view_product_mean', 'ecommerce_checkout_mean', 'ecommerce_conversion_mean'). 


Nota: Es posible realizar el cálculo de los promedios en una única operación sobre un dataframe, por lo que soluciones que realizan el cálculo del promedio de cada métrica por separado serán consideradas como menos eficientes.



In [1]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random as rd
%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [2]:
events = pd.DataFrame({ 
    'uid':[1,2,3,4,5,6,1,2,3,4,5,6,1,2,1,3,1,2,],
    'sessionid':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18],
    'event':['ecommerce.view-product', 'ecommerce.checkout', 'ecommerce.conversion', 'ecommerce.other', 'ecommerce.checkout', 'ecommerce.other','ecommerce.view-product', 'ecommerce.checkout', 'ecommerce.other', 'ecommerce.view-product', 'ecommerce.other', 'ecommerce.conversion','ecommerce.other', 'ecommerce.checkout', 'ecommerce.conversion', 'ecommerce.view-product', 'ecommerce.other', 'ecommerce.conversion'],
    'value':[1,6,3,4,5,12,8,2,5,7,5,6,1,2,9,4,2,6],
    'datetime':['2021-03-16','2021-03-17','2021-03-18','2021-03-17','2021-03-16','2021-03-15','2021-03-16','2021-03-17','2021-03-18','2021-03-17','2021-03-16','2021-03-15','2021-03-16','2021-03-17','2021-03-18','2021-03-17','2021-03-16','2021-03-15']

})
events = pd.DataFrame({ 
    'uid':rd.choices(range(1,4),k=18),
    'sessionid':range(1,19),
    'event':rd.sample(['ecommerce.view-product', 'ecommerce.checkout', 'ecommerce.conversion', 'ecommerce.other'],counts=[3,3,3,9],k=18),
    'value':rd.choices(range(1,12),k=18),
    'datetime': ['2021-03-15']*3+['2021-03-16']*3 + ['2021-03-17']*3 + ['2021-03-18']*9
}).astype({'datetime':'datetime64','event':'category'})
events.dtypes


uid                   int64
sessionid             int64
event              category
value                 int64
datetime     datetime64[ns]
dtype: object

In [3]:
users = pd.DataFrame({
    'uid':[1,2,3,4,5,6],
    'name':['Fer','Lucia','Alberto','Alan','Carla','Sofia']
})

In [4]:
events

,uid,sessionid,event,value,datetime
0,2,1,ecommerce.view-product,10,2021-03-15
1,1,2,ecommerce.checkout,8,2021-03-15
2,2,3,ecommerce.view-product,4,2021-03-15
3,1,4,ecommerce.other,5,2021-03-16
4,2,5,ecommerce.checkout,3,2021-03-16
5,2,6,ecommerce.other,7,2021-03-16
6,1,7,ecommerce.conversion,2,2021-03-17
7,1,8,ecommerce.conversion,3,2021-03-17
8,1,9,ecommerce.other,10,2021-03-17
9,2,10,ecommerce.other,8,2021-03-18


In [5]:
users

,uid,name
0,1,Fer
1,2,Lucia
2,3,Alberto
3,4,Alan
4,5,Carla
5,6,Sofia


In [6]:
events['sessions'] = 1
events_by_id = events.groupby('uid').count().reset_index()
events_by_id = events_by_id[['uid','sessions']] 
events_by_id

,uid,sessions
0,1,6
1,2,10
2,3,2


In [7]:
promedio_de_sesiones = events_by_id['sessions'].mean()
users_over_mean = events_by_id.loc[events_by_id['sessions']>promedio_de_sesiones,:]
users_over_mean

,uid,sessions
1,2,10


In [8]:
pd.merge(users_over_mean,users,on='uid',how='left')

,uid,sessions,name
0,2,10,Lucia


## Resolución de la profe

In [9]:
events['cant_sesiones']=events.groupby('uid')['sessionid'].transform('count')

In [10]:
events

,uid,sessionid,event,value,datetime,sessions,cant_sesiones
0,2,1,ecommerce.view-product,10,2021-03-15,1,10
1,1,2,ecommerce.checkout,8,2021-03-15,1,6
2,2,3,ecommerce.view-product,4,2021-03-15,1,10
3,1,4,ecommerce.other,5,2021-03-16,1,6
4,2,5,ecommerce.checkout,3,2021-03-16,1,10
5,2,6,ecommerce.other,7,2021-03-16,1,10
6,1,7,ecommerce.conversion,2,2021-03-17,1,6
7,1,8,ecommerce.conversion,3,2021-03-17,1,6
8,1,9,ecommerce.other,10,2021-03-17,1,6
9,2,10,ecommerce.other,8,2021-03-18,1,10


In [11]:
promedio=events[['uid','cant_sesiones']].drop_duplicates()['cant_sesiones'].mean()

In [12]:
promedio

6.0

In [13]:
events.loc[events['cant_sesiones']>promedio,'uid'].drop_duplicates().to_frame().merge(users.loc[1:3],how='inner')

,uid,name
0,2,Lucia


In [14]:
ptoA=events.loc[events['cant_sesiones']>promedio,'uid'].drop_duplicates().to_frame().merge(users)['name']

In [15]:
ptoA

0    Lucia
Name: name, dtype: object


b) Calcular el valor promedio de los eventos 'ecommerce.view-product', 'ecommerce.checkout', 'ecommerce.conversion' para aquellos usuarios que hayan realizado más de 25 (tomaremos 3 para el ejercicio) sesiones. Los resultados deben ser presentados en un dataframe con las siguientes columnas ('uid','ecommerce_view_product_mean', 'ecommerce_checkout_mean', 'ecommerce_conversion_mean'). 


In [16]:
users.loc[1:3]

,uid,name
1,2,Lucia
2,3,Alberto
3,4,Alan


In [17]:
events.head()

,uid,sessionid,event,value,datetime,sessions,cant_sesiones
0,2,1,ecommerce.view-product,10,2021-03-15,1,10
1,1,2,ecommerce.checkout,8,2021-03-15,1,6
2,2,3,ecommerce.view-product,4,2021-03-15,1,10
3,1,4,ecommerce.other,5,2021-03-16,1,6
4,2,5,ecommerce.checkout,3,2021-03-16,1,10


In [18]:
events_short= events.loc[(events['cant_sesiones']>3)&((events['event']=='ecommerce.view-product')|(events['event']=='ecommerce.checkout')|(events['event']=='ecommerce.conversion')),:]

In [19]:
events_short

,uid,sessionid,event,value,datetime,sessions,cant_sesiones
0,2,1,ecommerce.view-product,10,2021-03-15,1,10
1,1,2,ecommerce.checkout,8,2021-03-15,1,6
2,2,3,ecommerce.view-product,4,2021-03-15,1,10
4,2,5,ecommerce.checkout,3,2021-03-16,1,10
6,1,7,ecommerce.conversion,2,2021-03-17,1,6
7,1,8,ecommerce.conversion,3,2021-03-17,1,6
14,1,15,ecommerce.view-product,9,2021-03-18,1,6
16,2,17,ecommerce.checkout,1,2021-03-18,1,10


In [20]:
resultado=events_short.pivot_table(index='uid',columns='event',values='value',aggfunc='mean')

('uid','ecommerce_view_product_mean', 'ecommerce_checkout_mean', 'ecommerce_conversion_mean')


In [25]:
resultado.reset_index(inplace=True)
resultado

TypeError: 'fill_value=uid' is not present in this Categorical's categories

In [22]:
resultado[['uid','ecommerce.view-product', 'ecommerce.checkout', 'ecommerce.conversion']]

KeyError: "['uid'] not in index"

In [ ]:
resultado=resultado[['uid','ecommerce.view-product', 'ecommerce.checkout', 'ecommerce.conversion']]

In [ ]:
resultado.columns=['uid','ecommerce_view_product_mean', 'ecommerce_checkout_mean', 'ecommerce_conversion_mean']

In [ ]:
ptoB=resultado

In [ ]:
resultado.set_index('uid').T

uid,1,2
ecommerce_view_product_mean,4.50,nan
ecommerce_checkout_mean,nan,3.33
ecommerce_conversion_mean,9.00,6.00


In [ ]:
events.groupby('uid').mean().unstack()

               uid
sessionid      1                    10.60
               2                    10.50
               3                     9.33
               4                     7.00
               5                     8.00
               6                     9.00
value          1                     4.20
               2                     4.00
               3                     4.00
               4                     5.50
               5                     5.00
               6                     9.00
cant_sesiones  1                     5.00
               2                     4.00
               3                     3.00
               4                     2.00
               5                     2.00
               6                     2.00
dtype: float64

In [ ]:
events_short.drop_duplicates(['uid','event']).pivot(index='uid',columns='event',values='value')

event,ecommerce.checkout,ecommerce.conversion,ecommerce.view-product
uid,,,
1,nan,9.00,1.00
2,6.00,6.00,nan
